In [ ]:
# I have global temperature data from 1961 to 1990 in .nc format, called ‘combined_moving_avg_1961_1990.nc’. For each pixel, and for each calendar day, calculate the 90% percentile of daily maximum temperatures (Tmax) from 1961 to 1990, save the final result into a 2D .nc file called ‘heatwave_threshold_map.nc’

import xarray as xr
import numpy as np

# Load the NetCDF file
file_path = '/path/to/combined_moving_avg_1961_1990.nc'
ds = xr.open_dataset(file_path)

# Assume that 'tmax' is the name of the variable containing daily maximum temperatures
# If it's named differently, change 'tmax' to the correct variable name
tmax = ds['tmax']

# Ensure that time is in a datetime format
tmax['time'] = xr.cftime_range(start='1961-01-01', periods=len(tmax['time']), freq='D')

# Calculate the day of the year (1 to 366) for each time point, ignoring leap years
# Here, `tmax['time'].dt.dayofyear` generates a new dimension 'dayofyear'
tmax_dayofyear = tmax.groupby('time.dayofyear')

# Compute the 90th percentile for each pixel for each calendar day over all years
tmax_90_percentile = tmax_dayofyear.reduce(np.percentile, q=90, dim='time')

# The result will be a 3D array (dayofyear, lat, lon), so we'll average over the calendar days
# to generate a 2D map of heatwave thresholds (90th percentile of Tmax) for each pixel

# Save the final result as a new 2D NetCDF file
output_file = '/path/to/heatwave_threshold_map.nc'
tmax_90_percentile.to_netcdf(output_file)

print(f"Heatwave threshold map saved to {output_file}")
